# Introduction to SQL for Excel Users – Part 11: More CTEs & Windows

[Original post](https://www.daveondata.com/blog/introduction-to-sql-for-excel-users-part-11-more-ctes-windows/)

## RFM Analysis

NOTE – This post will divert a bit from the normal pattern. There will be no Excel coverage because the new concept (i.e., using multiple CTEs) is quite simple and I didn’t want the blog post be too long.

In this post I will show the power of SQL by implementing an old school, but still quite useful, technique – RFM analysis.

RFM analysis comes to us from the world of direct marketing (e.g., snail mail marketing).

RFM is primarily used as means of quantifying the value of customers along three vectors:

Recency – How recently has a customer made a purchase?
Frequency – How often does a customer make a purchase?
Monetary – How much does a customer spend on purchases?
Conceptually, the overall process for RFM analyses is quite simple. For each customer you calculate:

The elapsed time since their most recent purchase (the R)
How often purchases are made (the F)
Total amount spent (the M)
For steps 2 and 3 you can mix things up based on time scales (e.g., for all time, the current year, the last 6 months, etc.)

Once the raw RFM calculations are made you then rank each customer’s individual RFM scores on a scale from 1 to 10 using the following logic:

The top 10% of customers are assigned a score of 10
The next 10% of customers are assigned a score of 9
Repeat this process until the bottom 10% of customers are assigned a score of 1
This produces a ranking of 3 individual scores ranging from 1-10.

As you might imagine, your very best customers have scores of 10-10-10.

You worst customers have scores of 1-1-1.

You then spend your marketing dollars starting with the best customers and work your way down until you run out of money.

Easy peasy!

Also, it is worthy to note that this technique can be used for a broad range of scenarios.

For example, I have used the RFM idea to rank US zip codes based on data from the US Census Bureau.

## The Data

For this post I will be switching tables and using FactInternetSales.

Taking a peek in SQL Server Management Studio (SSMS):

![fact internet sales](11\factinternetsales.png)

FactInternetSales fits a particular data warehousing design pattern that makes various kinds of analyses using SQL quite easy.

For the purposes of a RFM analysis, the design of FactInternetSales means we have to work with groupings in the data.

Take the following snippet of data from FactInternetSales:

![fact internet sales groups](11\factinternetsalesgroups.png)

Notice in the data snippet ☝ that the value for CustomerKey (i.e., the unique ID for customers) are all the same.

However, notice that SalesOrderNumber has 3 distinct values:

- SO43793
- SO51522
- SO57418

Also notice the SalesOrderLineNumber column. This tells me that each row in FactInternetSales:

- Represents a single order line item (e.g., 47 widgets)…
- Purchased as part of a sales order (e.g., SO57418)…
- For a customer (e.g., 11000)

In terms of the RFM analysis, I will be working with two groups:

- I need to group OrderLineItemNumbers based on SalesOrderNumber
- I need to group SalesOrderNumbers based on CustomerKeys

Via the above groupings I can calculate the first round of numbers for each customer.

NOTE – Since I have not covered working with dates as of yet, this post will work with the **F** and the **M** only.

Don’t worry!

Dates are coming up shortly and I will revisit the **R** in an upcoming post.

## Grouping Sales Orders

In the last post I introduced CTEs.

CTEs are wildly useful in crafting most excellent SQL code.

I know from looking at the FactInternetSales data that I need to group sales orders and then use this group in the grouping by customers.

That means I’ll need to work directly with a virtual table of grouped sales orders.

Using a CTE is a great way to go in this scenario.

Here’s the first chunk of SQL:

In [ ]:
WITH CustomerSalesOrders AS
(
    SELECT FIS.CustomerKey
          ,FIS.SalesOrderNumber
          ,SUM(SalesAmount) AS SalesAmount
    FROM FactInternetSales FIS
    GROUP BY FIS.CustomerKey, FIS.SalesOrderNumber
)
SELECT *
FROM CustomerSalesOrders CSO
ORDER BY CSO.CustomerKey;

Awesome!

I now have sales order totals.

Notice that that the query ☝ above uses the SUM aggregate function.

Time to add the second grouping.

## Grouping Customers

Since I’ve wrapped the sales orders in a CTE, I can work with the query results directly as a named virtual table.

Here’s some SQL goodness that aggregates the sales order data to individual customers:

In [ ]:
WITH CustomerSalesOrders AS
(
    SELECT FIS.CustomerKey
          ,FIS.SalesOrderNumber
          ,SUM(SalesAmount) AS SalesAmount
    FROM FactInternetSales FIS
    GROUP BY FIS.CustomerKey, FIS.SalesOrderNumber
)
SELECT CSO.CustomerKey
      ,COUNT(*) AS SalesOrderCount
      ,SUM(CSO.SalesAmount) AS SalesAmount
FROM CustomerSalesOrders CSO
GROUP BY CSO.CustomerKey
ORDER BY CSO.CustomerKey;

The query ☝ makes use of the COUNT aggregate function.

Excellent.

Now I can use this last virtual table to perform the RFM scoring.

However, to do that I need to give this last virtual table a name…

## Multiple CTEs

SQL supports defining multiple CTEs in the same query.

Not surprisingly, analytics pros use this feature of SQL a lot.

And I mean a lot.

Here’s a general template of how to code this up:

```
WITH <CTE name #1 /> AS
(
     <CTE query />
),
<CTE name #2 /> AS
(
     <CTE query />
),
<CTE name X /> AS
(
     <CTE query />
)
SELECT <CTE columns />
FROM <CTE name />;
```

Using the above template, I can modify the SQL to be:


In [ ]:
WITH CustomerSalesOrders AS
(
    SELECT FIS.CustomerKey
          ,FIS.SalesOrderNumber
          ,SUM(SalesAmount) AS SalesAmount
    FROM FactInternetSales FIS
    GROUP BY FIS.CustomerKey, FIS.SalesOrderNumber
),
CustomerSalesOrderHistory AS
(
    SELECT CSO.CustomerKey
          ,COUNT(*) AS SalesOrderCount
          ,SUM(CSO.SalesAmount) AS SalesAmount
    FROM CustomerSalesOrders CSO
    GROUP BY CSO.CustomerKey
)
SELECT *
FROM CustomerSalesOrderHistory CSOH
ORDER BY CSOH.CustomerKey;

The above code produces the same results as in the last section.

Sweet!

Time to RFM.

## Using NTILE for RFM

To conduct the RFM analysis I will be using the NTILE window function.

NTILE is quite handy.

You provide NTILE an integer value (e.g., 10) and define a window.

NTILE then assigns a score appropriately.

The best way to understand NTILE is to see it in action:

In [ ]:
WITH CustomerSalesOrders AS
(
    SELECT FIS.CustomerKey
          ,FIS.SalesOrderNumber
          ,SUM(SalesAmount) AS SalesAmount
    FROM FactInternetSales FIS
    GROUP BY FIS.CustomerKey, FIS.SalesOrderNumber
),
CustomerSalesOrderHistory AS
(
    SELECT CSO.CustomerKey
          ,COUNT(*) AS SalesOrderCount
          ,SUM(CSO.SalesAmount) AS SalesAmount
    FROM CustomerSalesOrders CSO
    GROUP BY CSO.CustomerKey
)
SELECT CSOH.CustomerKey
      ,NTILE(10) OVER (ORDER BY CSOH.SalesOrderCount ASC) AS FrequencyScore
      ,NTILE(10) OVER (ORDER BY CSOH.SalesAmount ASC) AS MonetaryScore
FROM CustomerSalesOrderHistory CSOH
ORDER BY CSOH.CustomerKey;

Awwwww, yeah!

And if I wanted to find just my 10-10’s:

In [ ]:
WITH CustomerSalesOrders AS
(
    SELECT FIS.CustomerKey
          ,FIS.SalesOrderNumber
          ,SUM(SalesAmount) AS SalesAmount
    FROM FactInternetSales FIS
    GROUP BY FIS.CustomerKey, FIS.SalesOrderNumber
),
CustomerSalesOrderHistory AS
(
    SELECT CSO.CustomerKey
          ,COUNT(*) AS SalesOrderCount
          ,SUM(CSO.SalesAmount) AS SalesAmount
    FROM CustomerSalesOrders CSO
    GROUP BY CSO.CustomerKey
),
RFM AS
(
    SELECT CSOH.CustomerKey
          ,NTILE(10) OVER (ORDER BY CSOH.SalesOrderCount ASC) AS FrequencyScore
          ,NTILE(10) OVER (ORDER BY CSOH.SalesAmount ASC) AS MonetaryScore
    FROM CustomerSalesOrderHistory CSOH
)
SELECT *
FROM RFM FM
WHERE FM.FrequencyScore = 10 AND FM.MonetaryScore = 10
ORDER BY FM.CustomerKey;

Voila!

I should mention that you could produce the final results with less SQL using techniques I haven’t covered yet.

As usual, this example is contrived to suit my nefarious teaching plan. 😁

## T-SQL Window Functions

T-SQL’s window functions are wildly, wildly useful stuff for the analytics professional.

Be sure to check out Microsoft’s documentation for full details.

There also an awesome book dedicated to just this part of T-SQL’s goodness.

So far in this series I’ve covered just the ROW_NUMBER and NTILE window functions, there are others that are very useful:

- PERCENT_RANK
- LAG
- LEAD

Windows function – learn them, use them, 😍 them!

## The Learning Arc

The next post will be a bit of a departure.

Some folks are having issues with SSMS, so I am going to write a post with some tips.

Regular SQL goodness programming will resume after.

Stay healthy and happy data sleuthing!